In [60]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

docs_convert  = TextLoader("speech.txt")

text_load = docs_convert.load()
text_load



[Document(metadata={'source': 'speech.txt'}, page_content='Democracy and Zulifqar Ali Bhutto (Ex Prime minister of Pakistan)\n\nDemocracy is our Politics, Islam is our religion, Socialism is our Economy.\n\n The second elected Prime Minister of pakistan was Banzeer Bhutto daughter of zulifqar Ali Bhutto. \n \n\n This statement was dileverd by Ex Pakistani Prime Minister Zulifqar Ali Bhutto. \n\n\n He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.\n')]

In [61]:
splitter = CharacterTextSplitter(chunk_size= 200, chunk_overlap= 20)
text_splitter = splitter.split_documents(text_load)
text_splitter

[Document(metadata={'source': 'speech.txt'}, page_content='Democracy and Zulifqar Ali Bhutto (Ex Prime minister of Pakistan)\n\nDemocracy is our Politics, Islam is our religion, Socialism is our Economy.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The second elected Prime Minister of pakistan was Banzeer Bhutto daughter of zulifqar Ali Bhutto. \n \n\n This statement was dileverd by Ex Pakistani Prime Minister Zulifqar Ali Bhutto.'),
 Document(metadata={'source': 'speech.txt'}, page_content='He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.')]

In [62]:
embeddings = HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-v2")
db = FAISS.from_documents(text_splitter, embeddings)
db